In [1]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()

tr_x = tr_x * 1/255
te_x = te_x * 1/255

batch_size = 128

tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

170508288/170498071 [==============================] - 4s 0us/step


In [2]:
import effnet

model = effnet.efficientnet_lite_b0(input_shape = (32, 32, 3), include_top = False, weights = "imagenet")

flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
drop_out = tf.keras.layers.Dropout(0.5)(flatten)
dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
model = tf.keras.Model(model.input, prediction)

In [3]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(1e-4)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, optimizer = opt, metrics = metric)

In [4]:
model.fit(tr_data, validation_data = te_data, epochs = 10)

Epoch 1/10
391/391 [==============================] - 31s 39ms/step - loss: 1.7316 - sparse_categorical_accuracy: 0.3941 - val_loss: 1.2632 - val_sparse_categorical_accuracy: 0.5563
Epoch 2/10
391/391 [==============================] - 13s 34ms/step - loss: 1.2687 - sparse_categorical_accuracy: 0.5545 - val_loss: 1.0490 - val_sparse_categorical_accuracy: 0.6327
Epoch 3/10
391/391 [==============================] - 13s 34ms/step - loss: 1.0674 - sparse_categorical_accuracy: 0.6261 - val_loss: 0.9386 - val_sparse_categorical_accuracy: 0.6711
Epoch 4/10
391/391 [==============================] - 13s 34ms/step - loss: 0.9497 - sparse_categorical_accuracy: 0.6678 - val_loss: 0.8664 - val_sparse_categorical_accuracy: 0.6934
Epoch 5/10
391/391 [==============================] - 13s 34ms/step - loss: 0.8550 - sparse_categorical_accuracy: 0.7019 - val_loss: 0.8112 - val_sparse_categorical_accuracy: 0.7189
Epoch 6/10
391/391 [==============================] - 13s 34ms/step - loss: 0.7902 - spars

In [5]:
with open("model.json", mode = "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [6]:
with open("model.json", mode = "r") as file:
    model = tf.keras.models.model_from_json(file.read())
model.load_weights("model.h5")

In [7]:
loss = tf.keras.losses.sparse_categorical_crossentropy
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, metrics = metric)
model.evaluate(te_data)

79/79 [==============================] - 2s 15ms/step - loss: 0.6843 - sparse_categorical_accuracy: 0.7625


[0.6842713952064514, 0.762499988079071]